Jeromey Schwartz

HW2


801055747

https://github.com/JS-CTRL/RealTimeAI/tree/main/HW2

Problem 1

In [1]:
#importing libraries
%matplotlib inline
import numpy as np
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import pandas as pd
from torchvision import datasets, transforms
import torch.nn.functional as F
import torch.nn as nn
from collections import OrderedDict
import torch.optim as optim
torch.set_printoptions(edgeitems=2, linewidth=75)

In [2]:
#importing dataset
housing = pd.DataFrame(pd.read_csv("data/Housing.csv")) 
housing.head() 

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [3]:
#slicing out desired features from dataset
parameters = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking'] 
data_params = housing[parameters] 
t_u = housing[parameters] 
data_params.head() 
data_params.shape
housing_price=housing['price']
t_c=housing['price']
data_price=housing_price

In [4]:
t_u = torch.tensor(t_u.values,dtype=torch.float)
t_c = torch.tensor(t_c.values,dtype=torch.float)

In [5]:
#turning paramaters into a tensor
data = torch.tensor(data_params.values,dtype=torch.float)
data.shape

torch.Size([545, 5])

In [6]:
#converting cost to tensor
cost = torch.tensor(data_price.values,dtype=torch.float)
torch.unsqueeze(cost,0)
cost.shape

torch.Size([545])

In [7]:
#seperating data into 80/20 train/val split
n_samples = data.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

t_u_train= data[train_indices]
t_c_train= cost[train_indices]
t_u_val = data[val_indices]
t_c_val = cost[val_indices]



t_cn_val = F.normalize(t_c_val,dim=0)
t_cn_train= F.normalize(t_c_train,dim=0)

t_un_val = F.normalize(t_u_val,dim=0)
t_un_train= F.normalize(t_u_train,dim=0)

In [8]:
import datetime

def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()



def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train) # <1>
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val) # <1>
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()
            
        if epoch == 1  or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}, Validation loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train.item(), loss_val.item()))

In [9]:
#First model with following hidden layer size..
#    1st hidden layer = 8

seq_model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(5, 8)),
    ('hidden_activation', nn.Tanh()),
    ('output_linear', nn.Linear(8, 1))
]))

optimizer = optim.SGD(seq_model.parameters(), lr=1e-2)



training_loop(
    n_epochs = 300, 
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val, 
    t_c_train = t_cn_train,
    t_c_val = t_cn_val)

C:\Users\Jeromey\anaconda3\envs\RealTime\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([436])) that is different to the input size (torch.Size([436, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Jeromey\anaconda3\envs\RealTime\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([109])) that is different to the input size (torch.Size([109, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


2022-03-04 22:30:15.312648 Epoch 1, Training loss 0.03358231484889984, Validation loss 0.02186567708849907
2022-03-04 22:30:15.318653 Epoch 10, Training loss 0.01732766255736351, Validation loss 0.009677144698798656
2022-03-04 22:30:15.324659 Epoch 20, Training loss 0.008351471275091171, Validation loss 0.003934876527637243
2022-03-04 22:30:15.330664 Epoch 30, Training loss 0.004090785048902035, Validation loss 0.0019294505473226309
2022-03-04 22:30:15.337670 Epoch 40, Training loss 0.0020764213986694813, Validation loss 0.0014788457192480564
2022-03-04 22:30:15.343676 Epoch 50, Training loss 0.001126956194639206, Validation loss 0.001609200844541192
2022-03-04 22:30:15.349681 Epoch 60, Training loss 0.0006804285221733153, Validation loss 0.0019061375642195344
2022-03-04 22:30:15.356687 Epoch 70, Training loss 0.0004707613552454859, Validation loss 0.00220731133595109
2022-03-04 22:30:15.362693 Epoch 80, Training loss 0.00037242475082166493, Validation loss 0.0024594722781330347
2022-0

In [10]:
seq_model

for price, param in seq_model.named_parameters():
    print(price, param.shape)

hidden_linear.weight torch.Size([8, 5])
hidden_linear.bias torch.Size([8])
output_linear.weight torch.Size([1, 8])
output_linear.bias torch.Size([1])


In [11]:
list(seq_model.parameters())

[Parameter containing:
 tensor([[-0.0997,  0.2240,  0.0728, -0.1972,  0.3285],
         [-0.0045,  0.3910, -0.0107,  0.2389,  0.2687],
         [-0.4047,  0.1795, -0.1614,  0.3065,  0.0596],
         [ 0.4354, -0.3719,  0.0869, -0.3118, -0.4350],
         [ 0.0150, -0.2132,  0.3208, -0.2917, -0.1347],
         [-0.2949,  0.3523, -0.3177, -0.3615,  0.3291],
         [ 0.4198, -0.4391, -0.4002,  0.2026, -0.2789],
         [-0.0358,  0.0134, -0.3117,  0.3830,  0.2179]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.3543,  0.0880, -0.0192, -0.3314, -0.2405,  0.2807,  0.0261,
          0.4517], requires_grad=True),
 Parameter containing:
 tensor([[ 0.1870,  0.2947,  0.2859,  0.1813,  0.0051,  0.0104,  0.2129,
          -0.2578]], requires_grad=True),
 Parameter containing:
 tensor([0.2456], requires_grad=True)]

In [12]:
seq_model.output_linear.bias

Parameter containing:
tensor([0.2456], requires_grad=True)

In [13]:
#Second model with following hidden layer size..
#    1st hidden layer =8
#    2nd hidden layer = 80
#    3rd hidden layer = 20
seq_model = nn.Sequential(
     nn.Linear(5, 8),
     nn.Tanh(),
     nn.Linear(8, 80),
     nn.Tanh(),
     nn.Linear(80, 20),
     nn.Tanh(),
     nn.Linear(20, 1))

optimizer = optim.SGD(seq_model.parameters(), lr=1e-2)



training_loop(
    n_epochs = 300, 
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val, 
    t_c_train = t_cn_train,
    t_c_val = t_cn_val)

2022-03-04 22:30:15.562875 Epoch 1, Training loss 0.022394826635718346, Validation loss 0.03859438747167587
2022-03-04 22:30:15.574886 Epoch 10, Training loss 0.005526605527848005, Validation loss 0.014969206415116787
2022-03-04 22:30:15.586896 Epoch 20, Training loss 0.001343531534075737, Validation loss 0.007277234457433224
2022-03-04 22:30:15.598907 Epoch 30, Training loss 0.0005030253669247031, Validation loss 0.004867547191679478
2022-03-04 22:30:15.611919 Epoch 40, Training loss 0.0003350460610818118, Validation loss 0.003998683299869299
2022-03-04 22:30:15.623930 Epoch 50, Training loss 0.00030157214496284723, Validation loss 0.003652431769296527
2022-03-04 22:30:15.636942 Epoch 60, Training loss 0.0002949087065644562, Validation loss 0.0035062336828559637
2022-03-04 22:30:15.647952 Epoch 70, Training loss 0.00029358058236539364, Validation loss 0.0034426480997353792
2022-03-04 22:30:15.659963 Epoch 80, Training loss 0.0002933134092018008, Validation loss 0.00341458129696548
202

In [14]:
list(seq_model.parameters())

[Parameter containing:
 tensor([[ 0.3680,  0.0168,  0.3054, -0.1204, -0.2191],
         [-0.2350,  0.1064,  0.3124, -0.2980,  0.1009],
         [-0.2221, -0.3652, -0.3516,  0.2610, -0.0431],
         [ 0.0077, -0.0746, -0.2346,  0.1173,  0.0850],
         [ 0.3726,  0.2979, -0.1452,  0.0606,  0.2428],
         [-0.0017, -0.0534, -0.3270, -0.4190, -0.2532],
         [-0.3592,  0.0577,  0.3077,  0.0554,  0.4413],
         [ 0.3124,  0.3727,  0.2222,  0.1986, -0.3629]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.1436,  0.1177, -0.4347,  0.1773,  0.0767, -0.1565, -0.3452,
          0.1435], requires_grad=True),
 Parameter containing:
 tensor([[ 0.2654, -0.0615,  0.1266,  0.1784,  0.1105,  0.0307, -0.0719,
           0.3376],
         [ 0.0535,  0.2648, -0.1629, -0.1363,  0.1376, -0.0902, -0.0154,
          -0.1303],
         [-0.2921,  0.3522, -0.1608, -0.2349,  0.0372,  0.3215,  0.3068,
          -0.1119],
         [-0.1137, -0.2207,  0.3421, -0.2972,  0.0821,  0.0741

In [15]:
seq_model.parameters

<bound method Module.parameters of Sequential(
  (0): Linear(in_features=5, out_features=8, bias=True)
  (1): Tanh()
  (2): Linear(in_features=8, out_features=80, bias=True)
  (3): Tanh()
  (4): Linear(in_features=80, out_features=20, bias=True)
  (5): Tanh()
  (6): Linear(in_features=20, out_features=1, bias=True)
)>

In [16]:
seq_model

Sequential(
  (0): Linear(in_features=5, out_features=8, bias=True)
  (1): Tanh()
  (2): Linear(in_features=8, out_features=80, bias=True)
  (3): Tanh()
  (4): Linear(in_features=80, out_features=20, bias=True)
  (5): Tanh()
  (6): Linear(in_features=20, out_features=1, bias=True)
)

In [17]:
[param.shape for param in seq_model.parameters()]

[torch.Size([8, 5]),
 torch.Size([8]),
 torch.Size([80, 8]),
 torch.Size([80]),
 torch.Size([20, 80]),
 torch.Size([20]),
 torch.Size([1, 20]),
 torch.Size([1])]

Problem 2

In [18]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [19]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [20]:
#importing cifar10 data and loading the training and validation sets 
from torchvision import datasets, transforms
data_path = 'data/'

cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [21]:
cifar10

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: data/
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.4915, 0.4823, 0.4468), std=(0.247, 0.2435, 0.2616))
           )

In [22]:
cifar10_val

Dataset CIFAR10
    Number of datapoints: 10000
    Root location: data/
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.4915, 0.4823, 0.4468), std=(0.247, 0.2435, 0.2616))
           )

In [23]:
#Function to print out last models parameter count, training accuracy and validation accuracy
def model_summary():
    print(model)
    print("Parameter Count:" , sum([p.numel() for p in model.parameters()]))

    correct = 0
    total = 0

    with torch.no_grad():
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)  # <1>
            labels = labels.to(device=device)
            batch_size = imgs.shape[0]
            outputs = model(imgs.view(batch_size,-1))
            _, predicted = torch.max(outputs, dim=1)
            total += labels.shape[0]
            correct += int((predicted == labels).sum())

    print("Training Accuracy: %f" % (correct / total))

    correct = 0
    total = 0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs = imgs.to(device=device)  # <1>
            labels = labels.to(device=device)
            batch_size = imgs.shape[0]
            outputs = model(imgs.view(batch_size,-1))
            _, predicted = torch.max(outputs, dim=1)
            total += labels.shape[0]
            correct += int((predicted == labels).sum())

    print("Validation Accuracy: %f" % (correct / total))

In [24]:
#Seperating training and validation into batches for faster processing
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=50000,
                                           shuffle=True)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=10000,
                                         shuffle=True)

In [25]:
#setting device to GPU if applicable 
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
#device =torch.device('cpu') #uncomment if you want to force device=cpu
print(f"Training on device {device}.")

Training on device cuda.


In [26]:
#First model, 1 hidden layer with size of 512
import torch
import torch.nn as nn
import torch.optim as optim
import datetime

model1 = nn.Sequential(
            nn.Linear(3072, 512),
            nn.Tanh(),
            nn.Linear(512, 10),
            nn.LogSoftmax(dim=1)).to(device)

model=model1

learning_rate = 1e-2

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.NLLLoss().to(device)

n_epochs = 300

for epoch in range(n_epochs):
    epoch=epoch+1#added just to format print epoch 1,10,20...200 later on
    for imgs, labels in train_loader:
        imgs = imgs.to(device=device)  # <1>
        labels = labels.to(device=device)
        
        batch_size = imgs.shape[0]
        out = model(imgs.view(batch_size,-1))
        loss = loss_fn(out, labels)
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if epoch == 1  or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss))

2022-03-04 22:30:28.634531 Epoch 1, Training loss 2.3361594676971436
2022-03-04 22:31:50.483109 Epoch 10, Training loss 2.120896816253662
2022-03-04 22:33:22.034329 Epoch 20, Training loss 2.0459649562835693
2022-03-04 22:34:54.739624 Epoch 30, Training loss 2.0012950897216797
2022-03-04 22:36:27.490730 Epoch 40, Training loss 1.969862937927246
2022-03-04 22:38:00.440695 Epoch 50, Training loss 1.9460210800170898
2022-03-04 22:39:33.588325 Epoch 60, Training loss 1.9270731210708618
2022-03-04 22:41:06.959728 Epoch 70, Training loss 1.9115160703659058
2022-03-04 22:42:40.337905 Epoch 80, Training loss 1.8984037637710571
2022-03-04 22:44:13.963424 Epoch 90, Training loss 1.8871265649795532
2022-03-04 22:45:47.118019 Epoch 100, Training loss 1.8772573471069336
2022-03-04 22:47:20.554650 Epoch 110, Training loss 1.8684967756271362
2022-03-04 22:48:53.946121 Epoch 120, Training loss 1.8606220483779907
2022-03-04 22:50:27.495988 Epoch 130, Training loss 1.8534743785858154
2022-03-04 22:52:01

KeyboardInterrupt: 

In [27]:
#Using predifined function(self explanitory)
model_summary()

Sequential(
  (0): Linear(in_features=3072, out_features=512, bias=True)
  (1): Tanh()
  (2): Linear(in_features=512, out_features=10, bias=True)
  (3): LogSoftmax(dim=1)
)
Parameter Count: 1578506
Training Accuracy: 0.384240
Validation Accuracy: 0.381600


In [28]:
#Third model with following hidden layer size..
#    1st hidden layer =4096
#    2nd hidden layer = 2048
#    3rd hidden layer = 1024
import torch
import torch.nn as nn
import torch.optim as optim
import datetime

model3 = nn.Sequential(
            nn.Linear(3072, 4096),
            nn.Tanh(),
            nn.Linear(4096, 1024),
            nn.Tanh(),
            nn.Linear(1024, 64),
            nn.Tanh(),
            nn.Linear(64, 10),
            nn.LogSoftmax(dim=1)).to(device)

model=model3
learning_rate = 1e-2

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.NLLLoss().to(device)

n_epochs = 300

for epoch in range(n_epochs):
    epoch=epoch+1#added just to format print epoch 1,10,20...200 later on
    for imgs, labels in train_loader:
        imgs = imgs.to(device=device)  # <1>
        labels = labels.to(device=device)
        
        batch_size = imgs.shape[0]
        out = model(imgs.view(batch_size,-1))
        loss = loss_fn(out, labels)
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss))

2022-03-04 23:05:23.317439 Epoch 1, Training loss 2.3034963607788086
2022-03-04 23:06:47.996251 Epoch 10, Training loss 2.2259438037872314
2022-03-04 23:08:21.737090 Epoch 20, Training loss 2.1753599643707275
2022-03-04 23:09:55.573467 Epoch 30, Training loss 2.1410210132598877
2022-03-04 23:11:29.262195 Epoch 40, Training loss 2.1146578788757324
2022-03-04 23:13:02.978067 Epoch 50, Training loss 2.0929579734802246
2022-03-04 23:14:36.594106 Epoch 60, Training loss 2.074371814727783
2022-03-04 23:16:10.431212 Epoch 70, Training loss 2.0580615997314453
2022-03-04 23:17:44.249236 Epoch 80, Training loss 2.0435187816619873
2022-03-04 23:19:18.108781 Epoch 90, Training loss 2.0303919315338135
2022-03-04 23:20:51.776658 Epoch 100, Training loss 2.018437385559082
2022-03-04 23:22:25.648865 Epoch 110, Training loss 2.0074706077575684
2022-03-04 23:23:59.269061 Epoch 120, Training loss 1.9973422288894653
2022-03-04 23:25:33.169539 Epoch 130, Training loss 1.987939476966858
2022-03-04 23:27:07.

KeyboardInterrupt: 

In [29]:
#Using predifined function(self explanitory)
model_summary()

Sequential(
  (0): Linear(in_features=3072, out_features=4096, bias=True)
  (1): Tanh()
  (2): Linear(in_features=4096, out_features=1024, bias=True)
  (3): Tanh()
  (4): Linear(in_features=1024, out_features=64, bias=True)
  (5): Tanh()
  (6): Linear(in_features=64, out_features=10, bias=True)
  (7): LogSoftmax(dim=1)
)
Parameter Count: 16848586
Training Accuracy: 0.327920
Validation Accuracy: 0.327900
